# Spark Homework Assignments

In [1]:
import itertools as it
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import scipy.stats as st
# import scipy.misc
# import scipy.optimize as op
# import scipy.io
import random
from IPython.display import display, Math, Latex
import os
os.chdir('%s/courses/coursera_bigdata/course2' % os.getenv('DST'))
pwd = os.getcwd()
print(pwd)

# plotting options
%matplotlib inline
np.set_printoptions(linewidth=250)
plt.rc('font'  , size=18)
plt.rc('figure', figsize=(10, 8))
plt.rc('axes'  , labelsize=22)
plt.rc('legend', fontsize=16)

np.set_printoptions(precision=3)
plt.rc('figure', figsize=(10, 8))

/Users/rwk7t/Development/dst/courses/coursera_bigdata/course2


In [2]:
execfile('%s/python/pyspark/shell.py'%os.environ['SPARK_HOME'])

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.5.2
      /_/

Using Python version 2.7.10 (default, Jul 13 2015 12:18:59)
SparkContext available as sc, HiveContext available as sqlContext.


## Assignment 1

join data from two different file types

### load datasets

In [3]:
file1 = sc.textFile("file:%s/join1_file1.txt"%pwd)
file1.collect()

[u'able,991', u'about,11', u'burger,15', u'actor,22']

In [4]:
file2 = sc.textFile("file:%s/join1_file2.txt"%pwd)
file2.collect()

[u'Jan-01 able,5',
 u'Feb-02 about,3',
 u'Mar-03 about,8',
 u'Apr-04 able,13',
 u'Feb-22 actor,3',
 u'Feb-23 burger,5',
 u'Mar-08 burger,2',
 u'Dec-15 able,100']

### mapper for file 1

In [5]:
def split_file1(line):
    # split the input line in word and count on the comma
    word, count = line.strip().split(',', 1)
    count = int(count)
    return (word, count)

In [6]:
split_file1('ryan,7777')

('ryan', 7777)

In [7]:
file1_data = file1.map(split_file1)
file1_data

PythonRDD[4] at RDD at PythonRDD.scala:43

In [8]:
file1_data.collect()

[(u'able', 991), (u'about', 11), (u'burger', 15), (u'actor', 22)]

### mapper for file 2

In [9]:
def split_file2(line):
    date, other = line.split(' ', 1)
    word, count = line[len(date):].strip().split(',', 1)
    return (word, '%s %s'%(date, count))

In [10]:
split_file2('Apr-17 Ryan,38')

('Ryan', 'Apr-17 38')

In [11]:
file2_data = file2.map(split_file2)
file2_data

PythonRDD[5] at RDD at PythonRDD.scala:43

In [12]:
file2_data.collect()

[(u'able', u'Jan-01 5'),
 (u'about', u'Feb-02 3'),
 (u'about', u'Mar-03 8'),
 (u'able', u'Apr-04 13'),
 (u'actor', u'Feb-22 3'),
 (u'burger', u'Feb-23 5'),
 (u'burger', u'Mar-08 2'),
 (u'able', u'Dec-15 100')]

### run join

In [13]:
file2_joined_file1 = file2_data.join(file1_data)
file2_joined_file1

PythonRDD[13] at RDD at PythonRDD.scala:43

In [14]:
file2_joined_file1.collect()

[(u'able', (u'Jan-01 5', 991)),
 (u'able', (u'Apr-04 13', 991)),
 (u'able', (u'Dec-15 100', 991)),
 (u'burger', (u'Feb-23 5', 15)),
 (u'burger', (u'Mar-08 2', 15)),
 (u'about', (u'Feb-02 3', 11)),
 (u'about', (u'Mar-03 8', 11)),
 (u'actor', (u'Feb-22 3', 22))]

## Quiz 3

In [15]:
invalid = sc.accumulator(0)
def count_invalid(element):
    try:
        int(element)
    except:
        invalid.add(1)

In [16]:
d = sc.parallelize(["3", "23", "S", "99", "TT"]).foreach(count_invalid)

In [17]:
invalid.value

2

## Assignment 2

### do it with Dataframes first

In [18]:
df_gennumA = pd.read_csv('join2_gennumA.txt', header=None, names=['show', 'views'])
df_gennumB = pd.read_csv('join2_gennumB.txt', header=None, names=['show', 'views'])
df_gennumC = pd.read_csv('join2_gennumC.txt', header=None, names=['show', 'views'])
df_gennum  = pd.concat([df_gennumA, df_gennumB, df_gennumC], copy=True)
for df in [df_gennumA, df_gennumB, df_gennumC]:
    del(df)
df_gennum.head(3)

,show,views
0,Hourly_Sports,21
1,PostModern_Show,38
2,Surreal_News,73


In [19]:
df_views = df_gennum.groupby('show', as_index = False).sum()
del(df_gennum)
df_views.head(3)

,show,views
0,Almost_Cooking,49501
1,Almost_Games,49237
2,Almost_News,46592


In [20]:
df_genchanA = pd.read_csv('join2_genchanA.txt', header=None, names=['show', 'chan'])
df_genchanB = pd.read_csv('join2_genchanB.txt', header=None, names=['show', 'chan'])
df_genchanC = pd.read_csv('join2_genchanC.txt', header=None, names=['show', 'chan'])
df_genchan  = pd.concat([df_genchanA, df_genchanB, df_genchanC], copy=True)
for df in [df_genchanA, df_genchanB, df_genchanC]:
    del(df)
df_genchan.head(3)

,show,chan
0,Hourly_Sports,DEF
1,Baked_News,BAT
2,PostModern_Talking,XYZ


#### join

In [21]:
df_join = df_genchan.merge(df_views, on='show', how='inner')
df_join.head(3)

,show,chan,views
0,Hourly_Sports,DEF,54050
1,Hourly_Sports,DEF,54050
2,Hourly_Sports,DEF,54050


### filter for ABC

In [22]:
df_abc = df_join.query("chan=='ABC'")
df_abc

,show,chan,views
13,Baked_News,ABC,47211
50,Almost_Show,ABC,50202
82,Dumb_Show,ABC,53824
121,Hourly_Cooking,ABC,54208
141,PostModern_News,ABC,50021
170,Hot_Show,ABC,54378
190,Surreal_Sports,ABC,46834
219,Baked_Games,ABC,51604
240,Loud_Games,ABC,49482
278,Dumb_Talking,ABC,103894


In [23]:
df_abc = df_join[df_join.chan=='ABC']
df_abc

,show,chan,views
13,Baked_News,ABC,47211
50,Almost_Show,ABC,50202
82,Dumb_Show,ABC,53824
121,Hourly_Cooking,ABC,54208
141,PostModern_News,ABC,50021
170,Hot_Show,ABC,54378
190,Surreal_Sports,ABC,46834
219,Baked_Games,ABC,51604
240,Loud_Games,ABC,49482
278,Dumb_Talking,ABC,103894


### filter for BAT

In [24]:
df_bat = df_join[df_join.chan=='BAT']
df_bat.head(3)

,show,chan,views
10,Hourly_Sports,BAT,54050
12,Baked_News,BAT,47211
21,Baked_News,BAT,47211


In [25]:
df_bat = df_join.query("chan=='BAT'")
df_bat.head(3)

,show,chan,views
10,Hourly_Sports,BAT,54050
12,Baked_News,BAT,47211
21,Baked_News,BAT,47211


In [26]:
df_join.groupby('chan').sum()

,views
chan,
ABC,1115974
BAT,5099141
BOB,2591062
CAB,3940862
CNO,3941177
DEF,8032799
MAN,6566187
NOX,2583583
XYZ,5208016


### Now try spark

#### read the views files

In [31]:
show_views_file = sc.textFile("file:%s/join2_gennumA.txt"%os.getcwd())
show_views_file.take(3)

[u'Hourly_Sports,21', u'PostModern_Show,38', u'Surreal_News,73']

In [32]:
def split_show_views(line):
    shows, views = line.strip().split(',',1)
    views = int(views)
    return (shows, views)

In [33]:
show_views = show_views_file.map(split_show_views)
show_views.take(3)

[(u'Hourly_Sports', 21), (u'PostModern_Show', 38), (u'Surreal_News', 73)]

#### read the channel files

In [35]:
show_channel_file = sc.textFile("file:%s/join2_genchanA.txt"%os.getcwd())
show_channel_file.take(3)

[u'Hourly_Sports,DEF', u'Baked_News,BAT', u'PostModern_Talking,XYZ']

In [36]:
def split_show_channel(line):
    show, channel = line.strip().split(',',1)
    return (show, channel)

In [37]:
show_channel = show_channel_file.map(split_show_channel)
show_channel.take(3)

[(u'Hourly_Sports', u'DEF'),
 (u'Baked_News', u'BAT'),
 (u'PostModern_Talking', u'XYZ')]

#### join the data

In [38]:
views_join_channel = show_views.join(show_channel)
views_join_channel.take(3)

[(u'Surreal_Cooking', (596, u'CNO')),
 (u'Surreal_Cooking', (1037, u'CNO')),
 (u'Surreal_Cooking', (890, u'CNO'))]

#### extract channel as key

In [39]:
def extract_channel_views(show_view_channel):
    show, other = show_view_channel
    views, channel = other
    return (channel, views)

In [40]:
channel_views = views_join_channel.map(extract_channel_views)
channel_views.take(3)

[(u'CNO', 596), (u'CNO', 1037), (u'CNO', 890)]

#### sum accross all channels

In [41]:
def sum_views(v1, v2):
    return (v1+v2)

In [42]:
total_views = channel_views.reduceByKey(sum_views)

In [43]:
total_views.take(3)

[(u'CNO', 233893), (u'XYZ', 224945), (u'BAT', 195990)]

#### filter for 'BAT'

In [44]:
bat_viewers = total_views.filter(lambda r : r[0]=='BAT').collect()
bat_viewers

[(u'BAT', 195990)]

In [45]:
df = pd.DataFrame(data=bat_viewers, columns=['chan', 'views'])
df

,chan,views
0,BAT,195990


In [46]:
df.views.to_csv('bat_viewers.txt', sep = ' ', index = False, header = None)